# Modelo do Time Ganhador

O notebook [modelo_time_ganhador.ipynb](./modelo_time_ganhador.ipynb) é o ponto de execução para a criação do modelo de predição para prever qual time ganhará uma partida. Todas as predições são relacionados a Série A do Campeonato Brasileiro, incluindo dados de partidas, jogadores, equipes e ligas. O objetivo é prever os resultados das partidas. Além disso, esse notebook também é responsável pela explicabilidade dos modelos desenvolvidos, utilizando a acurácia, recall e F1-Score como métricas de avaliação.


É necessário importar as bibliotecas e módulos que serão utilizados no projeto. Além das bibliotecas principais, como **Pandas** e **Matplotlib**, também são necessárias ferramentas para manipulação de dados, visualização e criação de modelos preditivos. Abaixo está a lista detalhada das importações utilizadas:

**Bibliotecas Importadas**

- **import_ipynb**: Biblioteca que permite importar outros notebooks como módulos, reutilizando as funções neles definidas.
- **pre_processing**: Contêm as funções desenvolvidas para o pré-processamento dos dados.
- **pandas**: Utilizada para manipulação de dados em DataFrames, possibilitando leitura, transformação e análise dos dados.
- **matplotlib.pyplot**: Utilizada para criação de gráficos e visualização de dados.
- **seaborn**: Biblioteca de visualização de dados baseada no Matplotlib, usada para criar gráficos estatísticos atrativos e informativos.

**Módulos Importados do Scikit-learn**

- **train_test_split**: Função utilizada para dividir o dataset em conjuntos de treino e teste. Isso permite treinar o modelo com uma parte dos dados e testar seu desempenho com outra.
- **RandomForestClassifier**: Um classificador de aprendizado supervisionado baseado em florestas aleatórias, utilizado para prever classes categóricas, como o vencedor de uma partida.
- **Metrics (accuracy_score, precision_score, recall_score, f1_score)**: Métricas usadas para avaliar a precisão e o desempenho de classificadores, como o RandomForestClassifier.


In [17]:
import import_ipynb
import pre_processing 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lime.lime_tabular import LimeTabularExplainer

## 1. Preparação dos Dados e Modelagem


#### 1.1. Preparação dos Dados

A segunda parte da execução do trabalho envolve a preparação dos dados para o processo de modelagem preditiva. Para isso, as tabelas dos times e das partidas foram unificadas em um único arquivo CSV, permitindo a união das informações em um único dataset. Para acessar esse arquivo é necessário entrar na pasta do drive com os dados do grupo, por meio do seguinte [link](https://drive.google.com/drive/folders/1F_FVjAKfDFCbjb2CBesHeuoV4ZP0tvLG?usp=sharing). Com o arquivo baixado e salvo na pasta [notebook](../notebooks/), a segunda parte de execução consiste na definição da coluna de rótulo que, neste caso, é a coluna `winner`, responsável por indicar o time vencedor da partida.

Após a separação da coluna rótulo, as demais colunas passaram pelo processo de tratamentos, incluindo normalização e limpeza de valores nulos ou não informativos, utilizando a função de pré-processamento implementada no [notebook de pré-processamento](./pre_processing.ipynb). Esse tratamento garante que os dados estejam limpos e normalizados adequadamente para a construção do modelos preditivos, além de garantir que a variável alvo (coluna `winner`) seja preservada para o treinamento e avaliação do modelo.

**Passo a Passo da Célula**

1. **Carregamento do Dataset Unificado (`teams_with_matches.csv`)**

   - **Descrição:** A função `pd.read_csv()` da biblioteca `pandas` é utilizada para carregar o arquivo `teams_with_matches.csv`, que contém a unificação dos dados dos times e das partidas.
   - **Resultado:** O dataset `teams_with_matches` é carregado e está disponível para ser manipulado.

2. **Separação da Coluna Rótulo (`winner`)**

   - **Descrição:** A coluna `winner`, que indica o time vencedor da partida, é separada do restante do dataset. A coluna `Unnamed: 0` também é removida.
   - **Resultado:** A variável alvo (rótulo) `winner` é armazenada separadamente na variável `label_column`, enquanto o restante do DataFrame `teams_with_matches` é limpo, excluindo as colunas indesejadas.

3. **Normalização e Pré-Processamento do Dataset**

   - **Descrição:** A função `pre_processing.pre_processing()` é aplicada ao dataset `teams_with_matches` para realizar uma série de operações de pré-processamento, incluindo:
     - **Normalização:** Os dados numéricos são escalonados utilizando o `StandardScaler` e o `MinMaxScaler`.
     - **Tratamento de Valores Faltantes e Colunas Não Informativas:** as colunas que contém apenas valores zero ou um vário valores nulos são removidas.
     - **Codificação de Variáveis Categóricas:** as variáveis categóricas são transformadas em valores numéricos usando `LabelEncoder`.
   - **Resultado:** O DataFrame `teams_with_matches` é atualizado com os dados tratados e normalizados, e várias listas são retornadas para armazenar informações sobre as colunas tratadas e removidas, os `LabelEncoders` e `Scalers` usados no processo.

4. **Reinserção da Coluna Rótulo (`winner`)**

   - **Descrição:** Após o processo de normalização e tratamento, a coluna rótulo `winner` é adicionada novamente ao DataFrame, garantindo que a variável alvo esteja presente no conjunto de dados final.
   - **Resultado:** A coluna `winner`, separada no início do processo, é reintegrada ao DataFrame tratado e normalizado.

5. **Exportação do DataFrame Final para CSV**
   - **Descrição:** O DataFrame final, contendo os dados normalizados e a coluna rótulo `winner`, é exportado para um arquivo CSV chamado `teams_with_matches_normalized.csv`. A opção `index=False` é utilizada para evitar a criação de uma coluna de índice no arquivo CSV.
   - **Resultado:** O arquivo `teams_with_matches_normalized.csv` é salvo no diretório, contendo o dataset tratado e pronto para a etapa de modelagem preditiva.

**Resultados Obtidos**

- **Dataset Limpo e Unificado**: O arquivo CSV teams_with_matches.csv, contendo informações dos times e partidas, foi preparado para a modelagem. A coluna rótulo winner foi separada e reinserida ao final do processo.
- **Normalização e Tratamento de Dados**: O pré-processamento garantiu que todas as colunas numéricas fossem normalizadas e que as colunas categóricas fossem transformadas em valores numéricos. Além disso, as colunas não informativas foram removidas.
- **Exportação de Dados Tratados**: O dataset final foi exportado como teams_with_matches_normalized.csv, contendo todas as variáveis necessárias para o processo de modelagem, com os dados limpos, normalizados e estruturados.


In [ ]:
# Carregar o CSV
teams_with_matches = pd.read_csv('teams_with_matches.csv')

# Separar a coluna rótulo (substitua 'label_column' pelo nome da sua coluna rótulo)
label_column = teams_with_matches['winner']
teams_with_matches = teams_with_matches.drop(columns=['winner', 'Unnamed: 0'], axis=1)

# Normalizar o DataFrame sem a coluna rótulo (normalização fictícia no exemplo)
teams_with_matches, columns_trate_teams_with_matches, columns_drop_zero_teams_with_matches, columns_drop_null_teams_with_matches, label_encoders_teams_matches, scalers_teams_matches = pre_processing.pre_processing(teams_with_matches)


# Adicionar a coluna rótulo de volta ao DataFrame após a normalização
teams_with_matches['winner'] = label_column

# Exportar o DataFrame final para CSV
teams_with_matches.to_csv('teams_with_matches_normalized.csv', index=False)



#### 1.2. Treinamento do Modelo para Prever "Qual Time Irá Ganhar?"

A inteligência Artificial (IA) tem como objetivo encontrar modelos computacionais para os processos humanos inteligentes, sendo considerada uma ciência cognitiva que utiliza entidades representacionais, como símbolos ou operações, encontrados entre um estímulo (entradas) e uma resposta (saída). Além disso, a IA investiga as possibilidades de combinar, comparar ou transformar essas representações.

Sendo utilizada para análise de dados, previsões, categorização de objetos, processamento de linguagem natural ou recomendações, a IA compreende um conjunto de tecnologias baseadas em _Machine Learning_ ou _deep learning_. Conforme apresentado pelo Google Cloud, em _machine learning_ os algortimos são treinados em dados ou não rotulados para fazer previsões ou categorizar informação, enquanto no _deep learning_ são utilizados redes neurais artificial para processar as informações, simulando um cérebro humano.

O _machine learning_ é um subconjunto da inteligência artificial que utiliza algoritmos para treinar os dados e obter resultados, sendo dividido nos três tipos de modelos de aprendizado abaixo:

1. **Aprendizado Supervisionado:** modelo que mapeia uma entrada específica para uma saída usando dados de treinamento rotulados.
2. **Aprendizado Não Supervisionado:** modelo que identifica padrões com base em dados não rotulados, além do resultado final não ser conhecido antecipadamente.
3. **Aprendizado por Reforço:** modelo que aprende com base em feedbacks, em que um "agente" aprende a executar uma tarefa definida por tentativa e erro, até que o desempenho esteja dentro de um intervalo desejável.

Como apresentado anteriormente, a inteligência artificial possui diversos usos e aplicações, sendo o principal deles a classificação. Os algoritmos de classificação são algoritmos de aprendizado supervisionado que tem como objetivo prever uma classe ou rótulo baseado em um varíavel de entrada. Esse algoritmo é treinado com uma base de dados, e um dos principais algortimos de classificação é o _Random Forest_. O _Random Forest_ é uma técnica que constrói uma coleção de árvores de decisão independentes, usando o voto da maioria ou a média das predições dessas árvores para gerar a predição final.

Considerando esses conceitos, para prever qual time irá ganhar uma partida, é preciso desenvolver um modelo de classificação baseado em _Random Forest_, conforme apresentado abaixo.


#### 1.2.1. Treinamento e Execução do Modelo Preditivo

**Objetivo da Célula:**

Esta seção tem como objetivo a criação, o treinamento e a execução do modelo preditivo para prever o time vencedor de uma partida com base nos dados pré-processados e nas variáveis selecionadas do dataset unificado. Sendo que, esse processo envolve a separação dos dados em conjuntos de treino e teste, seguido pelo treinamento do modelo e a realização de previsões.

**Passo a Passo do Código**

1. **Carregar o Dataset Unificado**
   - **Descrição:** O dataset tratado e normalizado `teams_with_matches` é carregado novamente e armazenado na variável `df`.
2. **Separação da Coluna Rótulo (`winner`)**

   - **Descrição:** A variável `y` armazena a coluna `winner`, que representa o rótulo a ser previsto pelo modelo, ou seja, o time vencedor de cada partida.

     - A variável `X` contém o restante das colunas (excluindo `winner`), que serão usadas como variáveis explicativas (ou "features") no processo de modelagem. O código `X = X.iloc[:, 32:]` seleciona as colunas a partir da 32ª, eliminando variáveis que possam ser redundantes ou não informativas para o modelo.

   - **Resultado:** O dataset é dividido em `X` (variáveis explicativas) e `y` (rótulo a ser previsto), garantindo que o modelo use apenas as variáveis necessárias para prever o vencedor.

3. **Divisão em Conjuntos de Treinamento e Teste**

   - **Descrição:** A função `train_test_split()` da biblioteca `sklearn` é usada para dividir os dados em dois subconjuntos: um de treinamento e outro de teste. O parâmetro `test_size=0.2` indica que 20% dos dados serão reservados para teste, enquanto 80% serão utilizados para treinar o modelo.
     - O parâmetro `random_state=0` garante que a divisão seja reproduzível em execuções subsequentes, mantendo a consistência dos resultados.
   - **Resultado:** Os dados são divididos em quatro conjuntos:
     - `X_treinamento` e `y_treinamento`: Conjuntos de dados para treinar o modelo.
     - `X_teste` e `y_teste`: Conjuntos de dados reservados para testar o desempenho do modelo.

4. **Treinamento do Modelo Preditivo (Random Forest)**

   - **Descrição:** O classificador de floresta aleatória (`RandomForestClassifier`) é inicializado com os seguintes parâmetros:
     - **random_state=1:** Garante a reprodutibilidade dos resultados.
     - **n_estimators=100:** Define o número de árvores na floresta (100 árvores neste caso).
     - **max_depth=45:** Limita a profundidade máxima de cada árvore, controlando a complexidade do modelo e prevenindo overfitting.
     - **max_leaf_nodes=25:** Limita o número máximo de nós-folha (leaf nodes) em cada árvore, ajudando a controlar a complexidade do modelo.
   - Após a inicialização, o método `fit()` é utilizado para treinar o modelo nos dados de treinamento (`X_treinamento` e `y_treinamento`), permitindo que o classificador aprenda as relações entre as variáveis explicativas e o rótulo.
   - **Resultado:** O modelo de floresta aleatória é treinado e está pronto para fazer previsões sobre novos dados.

5. **Execução das Previsões**
   - **Descrição:** O método `predict()` é utilizado para gerar previsões com base nos dados de teste (`X_teste`). O modelo treinado utiliza o conhecimento adquirido durante o treinamento para prever o vencedor de cada partida no conjunto de teste.
   - **Resultado:** As previsões geradas são armazenadas na variável `previsoes`, que contém as classes previstas (ou seja, os vencedores previstos) para o conjunto de teste.

**Resultados Obtidos**

Com a execução deste trecho de código, obtemos as seguintes realizações:

- **Separação de Dados:** O dataset foi dividido corretamente em variáveis explicativas (`X`) e variável alvo (`y`).
- **Divisão em Treino e Teste:** Os dados foram divididos em conjuntos de treino e teste, assegurando que o modelo seja avaliado em um subconjunto de dados que não foi visto durante o treinamento.

- **Treinamento do Modelo:** O modelo de floresta aleatória foi treinado com parâmetros otimizados para equilibrar a precisão e a complexidade do modelo, utilizando profundidade máxima e número de nós-folha controlados.


In [11]:
# Carregar o dataset novamente
df = teams_with_matches

# Separar o rótulo (resultado do vencedor)
y = df['winner']
X = df.drop(columns=['winner'])
X = X.iloc[:, 32:]
# Separar em treino e teste
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.2, random_state=0)

# modelo = RandomForestClassifier(random_state=1, n_estimators=100)
modelo = RandomForestClassifier(random_state=1, n_estimators=100, max_depth=45, max_leaf_nodes=25)
modelo.fit(X_treinamento, y_treinamento)

previsoes = modelo.predict(X_teste)

#### 1.2.2. Avaliação do Modelo Preditivo

**Objetivo da Célula:**

Após o treinamento e a execução das previsões com o modelo de Random Forest, é necessário avaliar o desempenho do modelo utilizando métricas de classificação. As métricas escolhidas incluem **Acurácia**, **Precisão**, **Recall** e **F1-score**, conforme apresentado abaixo.

**Passo a Passo do Código**

1. **Cálculo da Acurácia**

   - **Descrição:** A função `accuracy_score()` da biblioteca `sklearn` calcula a acurácia do modelo, que é a proporção de previsões corretas em relação ao total de previsões feitas. A acurácia mede o quão bem o modelo classificou corretamente o time vencedor em cada partida.
   - **Resultado:** A variável `accuracy` armazena o valor da acurácia, que será exibido para avaliar a performance geral do modelo.

2. **Cálculo da Precisão**

   - **Descrição:** A função `precision_score()` calcula a precisão do modelo, que mede a proporção de verdadeiros positivos (predições corretas de um vencedor) em relação a todas as predições positivas feitas (verdadeiros positivos + falsos positivos). Neste caso, o parâmetro `average=None` indica que a precisão será calculada para cada classe separadamente (times diferentes).
   - **Resultado:** A variável `precision` armazena a precisão para cada classe, permitindo analisar o quão bem o modelo acerta as previsões para cada time.

3. **Cálculo do Recall**

   - **Descrição:** A função `recall_score()` calcula o recall, que mede a proporção de verdadeiros positivos em relação ao total de exemplos reais positivos (verdadeiros positivos + falsos negativos). O parâmetro `average='weighted'` calcula o recall ponderado por cada classe, levando em conta a proporção de ocorrências de cada classe no conjunto de dados.
   - **Resultado:** A variável `recall` armazena o valor do recall ponderado, indicando o quão bem o modelo consegue identificar corretamente as classes positivas em todo o conjunto de teste.

4. **Cálculo do F1-Score**

   - **Descrição:** A função `f1_score()` calcula o F1-score, que é a média harmônica entre a precisão e o recall, balanceando ambas as métricas para fornecer uma visão geral da performance do modelo em casos de classes desbalanceadas. O parâmetro `average='weighted'` calcula o F1-score ponderado por classe.
   - **Resultado:** A variável `f1` armazena o F1-score ponderado, que resume o equilíbrio entre precisão e recall em uma única métrica.

5. **Exibição dos Resultados**
   - **Descrição:** O `print()` exibe as métricas calculadas (Acurácia, Precisão, Recall e F1-score) de forma clara e formatada. As métricas de recall e F1-score são arredondadas para duas casas decimais para facilitar a interpretação.
   - **Resultado:** As métricas de desempenho são exibidas, permitindo a avaliação geral do modelo em termos de capacidade de previsão correta do vencedor de uma partida.

**Resultados Obtidos**

- **Acurácia:** Indica a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. Uma acurácia alta sugere que o modelo está acertando a maioria das previsões.
- **Precisão:** Avalia quantas das previsões positivas feitas pelo modelo foram realmente corretas, sendo que os valores altos de precisão indicam que o modelo é eficaz em minimizar falsos positivos.
- **Recall:** Mede a capacidade do modelo de identificar corretamente todas as instâncias da classe positiva, ou seja o quanto ele é capaz de capturar os vencedores reais das partidas.
- **F1-Score:** Oferece uma visão equilibrada entre precisão e recall, sendo útil quando há desequilíbrio nas classes (por exemplo, quando algumas equipes vencem com mais frequência do que outras).

**Importância das Métricas**

As métricas calculadas oferecem uma visão detalhada do desempenho do modelo em diferentes aspectos:

- **Acurácia:** Avalia o desempenho geral.
- **Precisão e Recall:** Medem o equilíbrio entre falsos positivos e falsos negativos, oferecendo uma análise mais granular do comportamento do modelo.
- **F1-Score:** Fornece uma métrica única que combina precisão e recall, especialmente útil em problemas com classes desbalanceadas.


In [ ]:
accuracy = accuracy_score(y_teste, previsoes)
precision = precision_score(y_teste, previsoes, average=None)
recall = recall_score(y_teste, previsoes, average='weighted')
f1 = f1_score(y_teste, previsoes, average='weighted')

print(f"Acurácia: {accuracy:2f}\nPrecisão: {precision}\nRecall: {recall.round(2)}\nF1: {f1.round(2)}")

**Análise das Métricas do Modelo de Classificação (Previsão do Ganhador)**

Após treinar o **RandomForestClassifier** para prever o ganhador de uma partida, as métricas de avaliação foram calculadas para medir o desempenho do modelo em termos de acurácia, precisão, recall e F1-score. Essas métricas são importantes para entender a capacidade do modelo de fazer previsões corretas.

**Métricas Obtidas:**

**Acurácia:**

- **Descrição:** A acurácia é a proporção de previsões corretas entre o total de previsões feitas. Neste caso, o modelo conseguiu prever corretamente o ganhador em 54,55% dos casos.
- **Resultado:** O modelo acertou aproximadamente 54,5% das previsões.

**Precisão:**

- **Descrição:** A precisão mede a proporção de previsões corretas para cada classe, levando em consideração apenas as previsões positivas. Cada valor corresponde à precisão para uma classe específica:
  - **0.6842**: Precisão para a classe `0` (vitória do time da casa).
  - **0.2**: Precisão para a classe `1` (empate).
  - **0.4444**: Precisão para a classe `2` (vitória do time visitante).
- **Resultado:** O modelo foi mais preciso ao prever vitórias do time da casa, enquanto teve dificuldades ao prever empates.

**Recall:**

- **Descrição:** O recall mede a proporção de exemplos positivos de uma classe que foram corretamente identificados. Neste caso, o valor de **0.55** indica que o modelo conseguiu identificar corretamente 55% dos ganhadores (time da casa, empate ou time visitante).
- **Resultado:** O modelo conseguiu identificar corretamente 55% dos ganhadores de partidas no conjunto de teste.

**F1-Score:**

- **Descrição:** O F1-score é a média harmônica entre precisão e recall, fornecendo dados do desempenho do modelo em termos de ambas as métricas. O valor de **0.54** indica um equilíbrio entre precisão e recall, embora o desempenho geral ainda seja modesto.
- **Resultado:** O F1-score indica que o modelo tem um desempenho médio, com uma combinação equilibrada de precisão e recall.

**Interpretação Geral:**

- O modelo apresenta uma acurácia de **54,55%**, o que significa que ele conseguiu prever corretamente o ganhador de uma partida em pouco mais da metade dos casos.
- A precisão varia significativamente entre as classes, com o modelo sendo mais preciso ao prever vitórias do time da casa (**68,42%**) e menos preciso ao prever empates (**20%**).
- O recall e o F1-score indicam um desempenho médio do modelo, com um recall de **55%** e F1-score de **54%**, sugerindo que há espaço para melhorias na capacidade do modelo de prever o ganhador.


#### 1.2.3. Função: `prever_vencedor_desnormalizado`

**Descrição:** A função `prever_vencedor_desnormalizado` é usada para prever o vencedor de uma partida entre dois times, utilizando um modelo preditivo previamente treinado. A função reverte as normalizações numéricas e categóricas dos dados, filtra as estatísticas relevantes para os dois times e faz a previsão com base nessas estatísticas.

#### **Parâmetros:**

- `time_1`: Nome do time da casa.
- `time_2`: Nome do time visitante.
- `df`: O DataFrame com os dados das partidas e times, que passou pelo pré-processamento e normalização.
- `modelo`: O modelo preditivo treinado, neste caso um `RandomForestClassifier`.
- `label_encoders`: Um dicionário com os `LabelEncoders` utilizados para codificar as colunas categóricas no pré-processamento.
- `scalers`: Um dicionário com os `StandardScaler` utilizados para normalizar as colunas numéricas no pré-processamento.

**Retorno:**

- `vencedor`: Uma string que descreve o vencedor previsto da partida, que pode ser "Vitória do time da casa", "Vitória do time visitante" ou "Empate".

**Passo a Passo do Código:**

1. **Reverter a Normalização das Colunas Numéricas e Categóricas**

   - **Descrição:** A função `reverse_numerics_columns` reverte a normalização dos dados numéricos, e `reverse_categoricals_columns` faz o mesmo para as colunas categóricas, restaurando o DataFrame para o formato original antes do pré-processamento.
   - **Resultado:** O DataFrame `df_desnormalizado` contém os dados desnormalizados, prontos para serem utilizados no filtro das estatísticas dos times.

2. **Filtrar as Estatísticas dos Times**

   - **Descrição:** As estatísticas do time da casa e do time visitante são filtradas do DataFrame desnormalizado. As colunas com informações sobre o time visitante são removidas para o time da casa, assim como para o time visitante.
   - **Resultado:** As variáveis `stats_time_1` e `stats_time_2` armazenam as estatísticas dos times da casa e visitante, respectivamente.

3. **Concatenar as Estatísticas dos Dois Times**

   - **Descrição:** As estatísticas de ambos os times são concatenadas horizontalmente, criando um único DataFrame com as informações necessárias para fazer a previsão. As colunas duplicadas são removidas para garantir que o conjunto de dados não esteja redundante.
   - **Resultado:** A variável `confronto_stats` contém as estatísticas combinadas de ambos os times.

4. **Garantir a Ordem Correta das Colunas para o Modelo**

   - **Descrição:** A ordem das colunas no DataFrame de estatísticas é ajustada para corresponder à ordem utilizada durante o treinamento do modelo, garantindo que o modelo receba as informações na mesma estrutura utilizada durante o treino.
   - **Resultado:** As colunas de `confronto_stats` são reorganizadas para coincidir com as features esperadas pelo modelo.

5. **Prever o Vencedor da Partida**

   - **Descrição:** A função `predict()` do modelo preditivo é usada para fazer a previsão com base nas estatísticas do confronto. A previsão será uma classe numérica: `0` para vitória do time da casa, `1` para empate, ou `2` para vitória do time visitante.
   - **Resultado:** A variável `previsao` armazena o resultado previsto.

6. **Interpretar e Exibir o Resultado**
   - **Descrição:** A previsão numérica é traduzida em uma mensagem descritiva que indica o vencedor previsto da partida, ou se houve um empate.
   - **Resultado:** A variável `vencedor` contém a descrição do resultado previsto.

**Resultados Obtidos:**
A função `prever_vencedor_desnormalizado` produz uma previsão sobre o resultado de uma partida entre dois times com base nas estatísticas disponíveis. Sendo os seguintes resultados alcançados:

- **Reversão da Normalização:** A função reverteu corretamente as normalizações aplicadas durante o pré-processamento, recuperando os dados originais, tanto para as colunas numéricas quanto para as categóricas.
- **Filtragem das Estatísticas Relevantes:** A função foi capaz de filtrar corretamente as estatísticas do time da casa e do time visitante, utilizando apenas as colunas relevantes para a previsão.
- **Concatenação e Organização das Estatísticas:** As estatísticas de ambos os times foram concatenadas em um DataFrame, organizadas na mesma estrutura usada durante o treinamento do modelo preditivo.
- **Previsão do Resultado da Partida:** O modelo previu corretamente o resultado da partida entre os dois times. O resultado foi uma das três classes possíveis:

  - `0`: Vitória do time da casa.
  - `1`: Empate.
  - `2`: Vitória do time visitante.

- **Interpretação da Previsão:** A previsão numérica foi traduzida em uma mensagem entendível para o usuário.


In [ ]:
# Função para prever o vencedor entre dois times
def prever_vencedor_desnormalizado(time_1, time_2, df, modelo, label_encoders, scalers):
    # Reverter normalização numérica
    df_desnormalizado = pre_processing.reverse_numerics_columns(df, label_encoders)

    # Reverter normalização categórica apenas para as colunas especificadas
    df_desnormalizado = pre_processing.reverse_categoricals_columns(df_desnormalizado, scalers)

    # Verifique se as colunas 'home_team_name' e 'away_team_name' estão no DataFrame após desnormalização
    if 'home_team_name' not in df_desnormalizado.columns or 'away_team_name' not in df_desnormalizado.columns:
        raise KeyError("As colunas 'home_team_name' ou 'away_team_name' estão faltando no DataFrame.")

    # Filtrar as estatísticas do time "home", pegar a partir da coluna 32 e remover "(away)"
    stats_time_1 = df_desnormalizado[df_desnormalizado['home_team_name'] == time_1].iloc[:, 32:]
    stats_time_1 = stats_time_1.loc[:, ~stats_time_1.columns.str.contains(r'\(away\)', case=False)]
    
    # Garantir que estamos pegando apenas a primeira linha de estatísticas
    stats_time_1 = stats_time_1.iloc[0:1]  # Seleciona apenas a primeira linha

    # Filtrar as estatísticas do time "away", pegar a partir da coluna 32 e remover "(home)"
    stats_time_2 = df_desnormalizado[df_desnormalizado['away_team_name'] == time_2].iloc[:, 32:]
    stats_time_2 = stats_time_2.loc[:, ~stats_time_2.columns.str.contains(r'\(home\)', case=False)]
    
    # Garantir que estamos pegando apenas a primeira linha de estatísticas
    stats_time_2 = stats_time_2.iloc[0:1]  # Seleciona apenas a primeira linha

    if stats_time_1.empty or stats_time_2.empty:
        raise ValueError("Um dos times não foi encontrado no dataset.")

    # Concatenar as estatísticas dos dois times (apenas uma linha de cada time)
    confronto_stats = pd.concat([stats_time_1.reset_index(drop=True), stats_time_2.reset_index(drop=True)], axis=1)

    # Remover colunas duplicadas, se houver
    confronto_stats = confronto_stats.loc[:, ~confronto_stats.columns.duplicated()]

    # Garantir que as colunas estejam na mesma ordem que durante o treinamento
    if hasattr(modelo, 'feature_names_in_'):
        # Reordenar as colunas com base nas colunas usadas no treinamento do modelo
        confronto_stats = confronto_stats.reindex(columns=modelo.feature_names_in_, fill_value=0)

    # Prever o vencedor
    previsao = modelo.predict(confronto_stats)

    # Reverter o label para o nome do time com base na diferença de gols
    if previsao[0] == 0:
        vencedor = f"Vitória do time da casa: {time_1}"
    elif previsao[0] == 1:
        vencedor = "Empate"
    elif previsao[0] == 2:
        vencedor = f"Vitória do time visitante: {time_2}"
    
    return vencedor

team_1 = input('Time 1: ')
team_2 = input('Time 2: ')

# Exemplo de uso da função para prever vencedor
vencedor = prever_vencedor_desnormalizado(team_1, team_2, teams_with_matches.drop(columns=['winner']), modelo, label_encoders, scalers)
print(f"O vencedor previsto entre {team_1} e {team_2} é: {vencedor}")


#### 1.2.4. Explicabilidade com LIME

**Objetivo da Célula:**

Nesta seção, a explicabilidade do modelo é feita utilizando a técnica LIME (Local Interpretable Model-Agnostic Explanations). O objetivo do LIME é explicar de forma interpretável o porquê de uma previsão específica feita pelo modelo. Essa técnica permite que, para cada instância do conjunto de teste, seja possível identificar as características que mais influenciam a previsão realizada.

**Passo a Passo do Código**

1. **Criação do Explicador LIME**

   - **Descrição:** O `LimeTabularExplainer` é inicializado com os dados de treinamento, os nomes das colunas (características), e o nome da classe que o modelo está prevendo (`winner`), além de especificar que estamos lidando com um problema de classificação.
   - **Resultado:** Um explicador LIME que pode ser usado para gerar explicações locais das previsões do modelo.

2. **Seleção de uma Instância para Explicação**

   - **Descrição:** Aqui, a primeira instância do conjunto de teste é selecionada para ser explicada. O `iloc[0]` seleciona a primeira linha do conjunto de teste, e o `.values` converte a linha para um array de valores numéricos.
   - **Resultado:** A variável `instancia_explicar` contém a instância selecionada para explicação.

3. **Geração da Explicação para a Previsão**

   - **Descrição:** A função `explain_instance()` gera uma explicação para a previsão da instância selecionada, utilizando a função de previsão de probabilidades do modelo (`predict_proba`).
   - **Resultado:** A variável `explicacao` armazena a explicação detalhada sobre quais características influenciaram mais a previsão para aquela instância.

4. **Mostrar a Explicação no Jupyter Notebook**

   - **Descrição:** Exibe a explicação diretamente no notebook em formato gráfico. Esse gráfico ajuda a visualizar quais características influenciaram mais a previsão e em que direção (positiva ou negativa).
   - **Resultado:** A explicação é exibida no Jupyter Notebook.

5. **Exibir a Explicação em Gráfico de Barras**

   - **Descrição:** Gera um gráfico de barras usando `matplotlib` que visualiza a contribuição de cada característica para a previsão.
   - **Resultado:** Um gráfico de barras exibindo a influência das principais características na previsão da instância.

6. **Imprimir a Explicação em Formato de Lista**

   - **Descrição:** Exibe a explicação em formato de lista, onde cada item da lista é uma característica acompanhada de sua respectiva contribuição para a previsão.
   - **Resultado:** A lista das características mais importantes, com sua contribuição específica para a previsão, é impressa no console.

7. **Gráfico de Barras: Contribuição das 10 Características Mais Importantes**

   - **Descrição:** Exibe um gráfico de barras horizontal mostrando as 10 características mais importantes que influenciaram a previsão. O valor LIME indica o peso de cada característica na decisão do modelo.
   - **Resultado:** Um gráfico que destaca as características mais influentes e seus respectivos pesos.

8. **Gráfico de Interação entre Duas Características**
   - **Descrição:** Exibe um gráfico de dispersão mostrando a relação entre duas das principais características, colorido de acordo com a previsão do modelo. Isso ajuda a visualizar como as interações entre características influenciam a previsão.
   - **Resultado:** Um gráfico de dispersão que mostra a interação entre duas características e sua relação com a previsão do modelo.

**Resultados Obtidos**

- **Explicação Local da Previsão:** O LIME gera uma explicação detalhada sobre como o modelo chegou à previsão para uma instância específica, na qual as características mais importantes para a previsão são destacadas.
- **Visualizações Gráficas:** Os gráficos fornecem informações visuais sobre a contribuição de cada característica para a previsão, bem como as interações entre diferentes características.
- **Interpretação das Decisões do Modelo:** A explicação permite entender melhor como o modelo toma decisões, sendo útil para melhorar a interpretabilidade e aumentar a confiança no modelo, especialmente em situações críticas de negócio.


In [ ]:
# -------------------------------------------
# Explicabilidade com LIME
# -------------------------------------------

# Criar o explicador LIME
explainer = LimeTabularExplainer(
    training_data=X_treinamento.values,  # Passar os dados de treinamento
    feature_names=X_treinamento.columns,  # Nome das colunas
    class_names='winner',  # Nome das classes do modelo de classificação
    mode='classification'  # Especificar que o modelo é de classificação
)

# Escolher uma instância do conjunto de teste para explicar
instancia_explicar = X_teste.iloc[0].values  # Primeira instância do conjunto de teste

# Gerar a explicação para a previsão da instância selecionada
explicacao = explainer.explain_instance(
    data_row=instancia_explicar,  # A linha que queremos explicar
    predict_fn=modelo.predict_proba  # A função de previsão do modelo (probabilidades)
)

# 1. Mostrar a explicação em forma de gráfico no notebook (para Jupyter)
explicacao.show_in_notebook(show_all=False)

# 2. Exibir a explicação em forma de gráfico de barras no terminal
fig = explicacao.as_pyplot_figure()  # Exibe a contribuição das características para a previsão
plt.show()

# 3. Imprimir a explicação das características como lista
print(explicacao.as_list())  # Exibe a contribuição das características para a previsão

# Gráficos adicionais para explicar melhor o "winner"

# 4. Gráfico de barras: Importância das características para a instância explicada
contribuicoes = explicacao.as_map()[1]  # Mapeamento dos valores LIME
caracteristicas = [X_teste.columns[i] for i, _ in contribuicoes]
valores_lime = [v for _, v in contribuicoes]

# 5. Gráfico de dispersão: Relação entre a primeira característica e a previsão
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_teste.iloc[:, 0], y=modelo.predict_proba(X_teste)[:, 1])
plt.xlabel(X_teste.columns[0])
plt.ylabel('Probabilidade de Previsão ("Winner")')
plt.title(f'Relação entre {X_teste.columns[0]} e a Previsão de Winner')
plt.show()

# 6. Gráfico de densidade: Distribuição da primeira característica no conjunto de treinamento
plt.figure(figsize=(8, 6))
sns.kdeplot(X_treinamento.iloc[:, 0], shade=True, label='Treinamento', color='blue')
plt.axvline(x=instancia_explicar[0], color='red', linestyle='--', label='Instância Explicada')
plt.xlabel(X_teste.columns[0])
plt.title(f'Distribuição de {X_teste.columns[0]} no Conjunto de Treinamento vs Instância Explicada')
plt.legend()
plt.show()


# Gráficos adicionais para explicar melhor o "winner"

# 1. Gráfico de Barras: Contribuição das 10 Características mais importantes
contribuicoes = explicacao.as_map()[1]  # Mapeamento dos valores LIME
caracteristicas = [X_teste.columns[i] for i, _ in contribuicoes]
valores_lime = [v for _, v in contribuicoes]

plt.figure(figsize=(10, 6))
plt.barh(caracteristicas, valores_lime, color='skyblue')
plt.xlabel('Contribuição LIME')
plt.title('Top 10 Características mais importantes para a Previsão')
plt.gca().invert_yaxis()
plt.show()

# 2. Gráfico de Interação: Relação entre duas características principais
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_teste.iloc[:, 0], y=X_teste.iloc[:, 5], hue=modelo.predict(X_teste))
plt.xlabel(X_teste.columns[0])
plt.ylabel(X_teste.columns[5])
plt.title(f'Interação entre {X_teste.columns[0]} e {X_teste.columns[5]} para a Previsão de Winner')
plt.show()


**Análise da Explicabilidade do Modelo para Prever o Ganhador Utilizando o LIME**

Os resultados de explicabilidade do primeiro modelo, obtidos por meio das visualizações com LIME, ajudam a entender quais características foram mais influentes para a previsão de vitória, empate ou derrota.

1. **Gráfico de Barras - Explicação Local LIME**
   O primeiro gráfico de barras exibe as contribuições das características para a previsão da partida selecionada. As principais características que influenciaram a previsão incluem:

- **(home)\_points_2h_home**: Contribuiu de forma significativa para a previsão, indicando que a pontuação do time da casa no segundo tempo foi um fator importante.
- **(away)\_goals_conceded_per_match**: O número de gols sofridos pelo time visitante por partida também teve uma forte influência negativa na previsão.
- **(home)\_win_percentage**: A porcentagem de vitórias do time da casa mostrou um impacto moderado.

2. **Gráfico de Dispersão - Relação entre (home)\_matches_played e Previsão de Winner**
   Este gráfico mostra a relação entre a característica `(home)_matches_played` e a probabilidade de previsão do "Winner". No eixo Y, temos a previsão de vitória/empate/derrota, e no eixo X, o número de partidas jogadas pelo time da casa. Podemos ver que o número de partidas jogadas pela equipe da casa pode afetar a probabilidade de vitória, uma vez que há uma concentração de previsões ao redor de valores específicos de partidas jogadas.

3. **Gráfico de Densidade - Distribuição de (home)\_matches_played**
   Este gráfico de densidade compara a distribuição da característica `(home)_matches_played` no conjunto de treinamento com a instância que foi explicada, sendo que a linha vermelha pontilhada indica o valor dessa característica na instância explicada. Em resumo, este gráfico ajuda a entender se a instância explicada está dentro de um comportamento típico ou se é um caso fora do comum em relação ao conjunto de dados usado para treinar o modelo.

4. **Gráfico de Barras - Top 10 Características mais importantes para a Previsão**
   O gráfico de barras mostra as 10 características mais importantes para a previsão do modelo. As principais características incluem:

- **(home)\_points_2h_home** e **(away)\_goals_conceded_per_match**, que novamente são fatores dominantes.
- O gráfico também mostra características como **(home)\_win_percentage** e **(home)\_draw_percentage**, que reforçam a importância do histórico de desempenho dos times da casa e visitante para a previsão de vitória.

5. **Gráfico de Interação - (home)\_matches_played vs (home)\_wins_home**
   O gráfico de interação mostra a relação entre as características `(home)_matches_played` e `(home)_wins_home`, visualizando como a combinação dessas duas características afeta a previsão do vencedor. As diferentes cores representam diferentes classes de previsão (vitória do time da casa, empate ou vitória do time visitante), e a dispersão dos pontos mostra como essas variáveis interagem na definição do resultado da partida.

Essas visualizações fornecem uma compreensão dos fatores que o modelo considera mais importantes para determinar o resultado de uma partida. Entretanto, além de compreender os resultados também é relevante enteder as motivações das escolhas feitas para explicar o modelo, conforme exposto abaixo.

Considerando o contexto de modelos de aprendizado de máquina, como o _Random Forest_, é importante garantir que as previsões feitas possam ser explicadas de maneira compreensível, dessa forma, o _LIME (Local Interpretable Model-Agnostic Explanations)_ foi a técnica escolhida para fornecer essa explicação por conta das seguintes razões:

- **Explicabilidade Local:** O LIME permite explicar as previsões individuais de forma local, ou seja, fornece uma explicação para uma instância específica, sendo útil para entender como um conjunto específico de características contribuiu para o resultado da previsão de uma partida (vitória do time da casa, empate, vitória do visitante). Dessa forma, ao invés de fornecer uma explicação genérica para todo o modelo, o LIME permite ver o impacto de cada característica para cada previsão.

- **Modelo Agnóstico:** O LIME pode ser aplicado a qualquer tipo de modelo, como redes neurais, SVMs, ou no caso desenvolvido, Random Forest, isso oferece flexibilidade na explicação, independentemente do algoritmo utilizado. Então considerando que o LIME é independente do tipo de modelo ele foi escolhido para explicar o modelo desenvolvido.

- **Transparência e Compreensão de Características:** Ao aplicar o LIME, é possível entender quais características (features) são mais importantes para a previsão de uma partida. Sendo que, a explicação em termos de características ajuda a validar o modelo, garantindo que ele está tomando decisões baseadas em fatores que fazem sentido no domínio de previsão de resultados de partidas de futebol.

- **Interpretação Visual das Previsões:** Os gráficos gerados (barras, dispersão, densidade) fornecem uma maneira intuitiva de visualizar a contribuição de cada característica para a previsão. Em vez de apenas observar números, esses gráficos facilitam a interpretação para não-especialistas, permitindo que tanto stakeholders técnicos quanto não-técnicos entendam as decisões do modelo.

Após compreender quais foram os motivos que levaram ao uso do LIME para explicar o modelo, também é preciso entender quais as razões para a escolha dos gráficos exibidos, sendo que o gráfico de barras foi utilizado para visualizar o impacto de cada característica individualmente, dado que a separação das contribuições em barras permite observar quais características possuem maior impacto (positivo ou negativo) na previsão de um vencedor, ajudando a validar se o modelo está focando nas variáveis corretas, como gols sofridos ou porcentagem de vitórias.

Além do gráfico de barras, há o gráfico de dispersão que foi gerado para explorar a relação entre o número de partidas jogadas pela equipe da casa e a probabilidade de vitória/empate/derrota. Dessa forma, ele ajuda a identificar se há uma tendência de correlação entre essa variável e o resultado da partida. Logo, esse gráfico é útil para visualizar como a quantidade de partidas pode influenciar diretamente a probabilidade de vitória.

Somado a esses, há o gráfico de densidade que foi escolhido para comparar a distribuição do número de partidas jogadas no conjunto de dados de treinamento em relação à instância que foi explicada, de forma que seja possível verificar se a instância explicada está em uma área onde o modelo tem mais ou menos confiança, auxiliando na interpretação de como a distribuição de dados afeta a previsão.

Por fim, os dois últimos gráficos gerados são o gráfico de barras (Top 10 Características mais importantes) e o gráfico de interação (entre (home)\_matches_played vs (home)\_wins_home). O primeiro gráfico ajuda na visualização das características que mais influenciam a previsão de vitória. Enquanto no segundo gráfico a interação entre o número de partidas jogadas e o número de vitórias em casa ajuda a entender como duas características importantes afetam conjuntamente a previsão, permitindo explorar como a interação de diversas variáveis pode alterar as previsões feitas pelo modelo.

Por meio das análises feitas, é possível concluir que a utilização do LIME e a escolha dos gráficos de explicabilidade foram importantes para garantir o desempenho do modelo preditivo, sendo que a interpretação local fornecida pelo LIME, em conjunto com os gráficos de barras e dispersão, proporcionam forma de validar o modelo e entender suas previsões. Essas escolhas foram positivas, considerando que o público-alvo do projeto (técnicos de times e patrocinadores de jogadores) utilizam dados para tomarem decisões, logo um modelo preditivo com explicações relevantes pode promover mais confiança para uma tomada de decisão baseada nos seus resultados.
